In [28]:
import urllib
import re
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.io as sio
import requests

url = "https://zillow56.p.rapidapi.com/search"
houseNumberPattern = re.compile(r'\b[0-9]{1,6}\b')

price_lower = []
price_middle = []
price_higher = []
address = []
floorArea = []
type = []
bathroom_num = []
bedroom_num = []
livingroom_num = []
lastSoldYear = []
longitude = []
latitude = []
house_number = []

# propertyIDBegin = 383
# propertyIDEnd = 443
# baseVal = 7123-propertyIDBegin

In [5]:
propertyIdx = str(int(12002))
streetName = "Cheshire St, Norwalk, CA"

querystring = {"location":propertyIdx + " " + streetName}

headers = {
	"X-RapidAPI-Key": "76325c0abamshe24a0bd685d4898p1ed9b3jsn63784230fa57",
	"X-RapidAPI-Host": "zillow56.p.rapidapi.com"
}



In [7]:
response = requests.request("GET", url, headers=headers, params=querystring)
res = response.json()
res

{'address': {'city': 'Norwalk',
  'community': None,
  'neighborhood': None,
  'state': 'CA',
  'streetAddress': '12002 Cheshire St',
  'subdivision': None,
  'zipcode': '90650'},
 'attributionInfo': {'agentEmail': None,
  'agentLicenseNumber': None,
  'agentName': None,
  'agentPhoneNumber': None,
  'attributionTitle': None,
  'brokerName': None,
  'brokerPhoneNumber': None,
  'buyerAgentMemberStateLicense': None,
  'buyerAgentName': None,
  'buyerBrokerageName': None,
  'coAgentLicenseNumber': None,
  'coAgentName': None,
  'coAgentNumber': None,
  'lastChecked': None,
  'lastUpdated': None,
  'listingAgents': [],
  'listingAgreement': None,
  'listingOffices': [{'associatedOfficeType': 'listOffice',
    'officeName': None}],
  'mlsDisclaimer': None,
  'mlsId': None,
  'mlsName': None,
  'providerLogo': None,
  'trueStatus': None},
 'bathrooms': 2,
 'bedrooms': 3,
 'brokerageName': None,
 'building': None,
 'city': 'Norwalk',
 'contingentListingType': None,
 'countyFIPS': '06037',
 '

In [29]:
price_middle.append(int(res['zestimate']))
lowEst = int(round(res['zestimate'] * (100 - int(res['zestimateLowPercent'])) / 100000) * 1000)
highEst = int(round(res['zestimate'] * (100 + int(res['zestimateHighPercent'])) / 100000) * 1000)
price_lower.append(lowEst)
price_higher.append(highEst)

adStr = res['address']['streetAddress'] + ", " + res['address']['city'] + ", " + res['address']['zipcode']
address.append(adStr)

floorArea.append(res['livingArea'])
type.append(res['homeType'])
bathroom_num.append(res['bathrooms'])
bedroom_num.append(res['bedrooms'])
livingroom_num.append(float('nan'))
lastSoldYear.append(int(res['priceHistory'][0]['date'][0:4]))

longitude.append(res['longitude'])
latitude.append(res['latitude'])

house_number.append(int(houseNumberPattern.findall(adStr)[0]))

In [31]:
raw_data = pd.DataFrame({"Low": price_lower, "Estimated price": price_middle, "High": price_higher,
                        "Address": address, 'House number': house_number, 
                        "Longitude": longitude, "Latitude": latitude, "Last Sold": lastSoldYear,
                        "Floor Area": floorArea, "Property Type": type,
                        "Bedrooms": bedroom_num, "Bathrooms": bathroom_num, "Livingrooms": livingroom_num})
raw_data.to_csv("CA Geo data/"+res['address']['zipcode']+" price data Geo.csv", index=True, header=True)
raw_data

,Low,Estimated price,High,Address,House number,Longitude,Latitude,Last Sold,Floor Area,Property Type,Bedrooms,Bathrooms,Livingrooms
0,679000,714400,750000,"12002 Cheshire St, Norwalk, 90650",12002,-118.077545,33.890217,2012,1299,SINGLE_FAMILY,3,2,NaN
